In [1]:
import pandas as pd
import pickle
import csv
import os
import re
import emoji
from textblob import TextBlob
import nltk
from langdetect import detect
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix
import gensim
nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
## dic for contraction sub
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",   
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [39]:
## Cleaning methods
def clean_emoji(text):
    text=emoji.demojize(text)
    return text

def remove_url(text):
    text = ' '.join(re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"," ", text).split())
    return text


def remove_symbol(text):
    text = str.replace(text,',',' ')
    text = str.replace(text,'+',' ')
    text = str.replace(text,'=',' ')
    text = ' '.join(re.sub("(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})"," ", text).split())
    text = str.replace(text,"'",' ')
    text = str.replace(text,'"',' ')
    text = str.replace(text,'!',' ')
    text = str.replace(text,'^',' ')
    text = str.replace(text,'(',' ')
    text = str.replace(text,')',' ')
    text = str.replace(text,'%',' ')
    text = str.replace(text,'-',' ')
    text = str.replace(text,'_',' ')
    text = str.replace(text,'|',' ')
    text = str.replace(text,'.',' ')
    text = str.replace(text,':',' ')
    return text
    
## this is the optional function we can use to remove @ and other. Might not use it if we need to analyze retweets
## also set all to lower case
def optional_rm(text):
    all_words = text.split(' ')
    remove_words = []
    for each in all_words:
        if each == ' ' or each == '':
            continue
        if each[0] == '#' or each[0] == '@':
            remove_words.append(each)
    for word in remove_words:
        all_words.remove(word)
    text = ' '.join(all_words)
    
    text = text.split(' ')
    new_text = []
    for each in text:
        if(str.find(each,'http') != -1):
            continue
        
        if not each.isalnum():
            continue
        new_text.append(str.lower(each));
    text = ' '.join(new_text)
    
    return text
    
def is_english(text):
    try:
        if detect(text) == "en":
            return True
        else:
            return False
    except Exception as e:
        return True
        print(e)
    
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

    
def processing(text):
    #text=clean_emoji(text)
    #text=remove_url(text)
    #text = remove_symbol(text)
    text = optional_rm(text)
    return text

In [6]:
## Split the data into two parts, one for original tweet txt, another one for tweet txt
## drop na 
## remove the row which tweet_txt has no full text
def split_data(dataframe):
    df_og=dataframe[['og_tweet_txt', 'og_tweet_time', 'og_tweet_id', 'og_tweet_user_id',
       'og_tweet_user_name', 'og_tweet_user_desc', 'og_tweet_user_vrifd',
       'og_tweet_user_loc']]
    df_tweet=dataframe[['tweet_txt', 'user_loc', 'tweet_geo',
       'tweet_place', 'tweet_time', 'tweet_id', 'user_id', 'user_name',
       'tweet_likes', 'tweet_source', 'hashtags', 'user_acc_cr_time',
       'user_verified', 'user_total_tweets', 'user_followers']]
    dt_og=df_og.copy()
    dt_og=dt_og.dropna(axis=0,how='any')
    dt_og=dt_og[~dt_og['og_tweet_txt'].isin(['\u2026'])]  
    dt_tweet=df_tweet.copy()
    # dt_tweet=dt_tweet.dropna(axis=0,how='any')
    dt_tweet=dt_tweet[~dt_tweet['tweet_txt'].str.contains('\u2026')]
    
    
    ## text cleaning
#     dt_og['tweet_txt_clean']=dt_og['og_tweet_txt'].apply(lambda x: processing(x))
#     dt_tweet['tweet_txt_clean']=dt_tweet['tweet_txt'].apply(lambda x: processing(x))
    
#     dt_og = dt_og[dt_og["tweet_txt_clean"].apply(is_english)]
#     dt_tweet = dt_tweet[dt_tweet["tweet_txt_clean"].apply(is_english)]
    
    return dt_og, dt_tweet

In [4]:
## Import Data
def read_json_to_dataframe(filename):
    print(filename)
    #f = open(filename,encoding = 'utf-16').read()
    f1 = pd.read_json(filename, orient='records',lines=True,encoding = 'utf-16')
    return f1
def read_in(path):
    files = os.listdir(path)
    all_dfs = []
    print(files)
    for i in files:
        if 'json' in i:
            each_df = read_json_to_dataframe(path+i)
            all_dfs.append(each_df)
            
    return pd.concat(all_dfs, ignore_index=True)

all_df = read_in('C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/')
all_df.shape

['02-17-2021.json', '02-18-2021.json', '02-19-2021.json', '02-20-2021.json', '02-21-2021.json', '02-22-2021.json', '02-23-2021.json', '02-24-2021.json']
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-17-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-18-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-19-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-20-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-21-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-22-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-23-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-24-2021.json


(970708, 23)

In [36]:
## Get rid of the folded tweets
test_df = all_df.copy()
test_df=test_df[~test_df['og_tweet_txt'].str.endswith('\u2026',na=True)]
test_df=test_df[~((test_df['tweet_txt'].str.endswith('\u2026',na=True)) & (test_df['og_tweet_txt'].str.strip() == ""))]
# test_df=test_df.dropna(axis=0,how='any')
#test_df

In [41]:
def clean_txt(df):
    df["og_tweet_txt"] = df['og_tweet_txt'].apply(lambda x: processing(x))
    print('finish processing retweet')
    df['tweet_txt'] = df['tweet_txt'].apply(lambda x: processing(x))
    print('finish processing no retweet')
    #df[df["tweet_txt"].apply(is_english)]
    #df[df["og_tweet_txt"].apply(is_english)]
    return df

In [48]:
test_df = test_df.drop_duplicates(subset='tweet_txt', keep="last")
#test_df = test_df.drop_duplicates(subset='og_tweet_txt', keep="last")

KeyboardInterrupt: 

In [33]:
clean_df = clean_txt(test_df)

<ipython-input-26-edc904c48e08>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["og_tweet_txt"] = df['og_tweet_txt'].apply(lambda x: processing(x))


finish processing retweet


<ipython-input-26-edc904c48e08>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_txt'] = df['tweet_txt'].apply(lambda x: processing(x))


finish processing no retweet


In [42]:
clean_df = clean_txt(clean_df)

<ipython-input-41-983449fb880e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["og_tweet_txt"] = df['og_tweet_txt'].apply(lambda x: processing(x))


finish processing retweet
finish processing no retweet


<ipython-input-41-983449fb880e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_txt'] = df['tweet_txt'].apply(lambda x: processing(x))


In [47]:
clean_df

,og_tweet_txt,og_tweet_time,og_tweet_id,og_tweet_user_id,og_tweet_user_name,og_tweet_user_desc,og_tweet_user_vrifd,og_tweet_user_loc,tweet_txt,user_loc,...,tweet_id,user_id,user_name,tweet_likes,tweet_source,hashtags,user_acc_cr_time,user_verified,user_total_tweets,user_followers
3,,NaT,,,,,,,is anyone gonna mention biden forgot he receiv...,On a Lake,...,1362255917085986817,34742700,tinberry,0,Twitter Web App,,2009-04-23 21:51:32,False,1737,21
7,,NaT,,,,,,,just on presentation alone he should be put in...,"Saratoga Springs, NY",...,1362255933586485249,1218151375609180163,SML531974,0,Twitter for Android,,2020-01-17 12:41:56,False,13152,452
8,the spread of highly contagious coronavirus va...,2021-02-16 19:10:06,1361754747880022018,3108351,WSJ,Sign up for our newsletters and email alerts: ...,True,"New York, NY",rt the spread of highly contagious coronavirus...,None,...,1362255935285133313,738884622147289088,Lgmcruiser,0,Twitter for iPhone,,2016-06-04 00:06:36,False,14130,27
9,,NaT,,,,,,,claire pfizer only enrolled 5 ppl over the age...,World,...,1362255938292314115,796908792093020160,CarmenNasty2016,0,Twitter Web App,,2016-11-11 02:53:56,False,2238,39
10,,NaT,,,,,,,is this biden s uniqua,None,...,1362255938405588992,1291573086022639617,BotSatire,0,Twitter Web App,,2020-08-07 03:13:40,False,1185,153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970703,making you get tested for covid before you ent...,2021-02-25 01:41:54,1364752447831375873,1195136746524012544,TrishForTrump,Real Estate Broker...Investor...USA ...”Joe’s ...,False,"California, USA",rt making you get tested for covid before you ...,"Cyberdyne Systems, Inc.",...,1364847442609336325,2590012027,Algerwins,0,Twitter Web App,,2014-06-26 17:42:54,False,66110,5216
970704,whose coming partying face partying face party...,2021-02-24 22:19:00,1364701389692280835,83197153,Lyndonx,This is real and raw. Sharing experiences & li...,True,Zapped it,rt whose coming partying face partying face pa...,London,...,1364847445151219714,3003029949,ThemysciranJZ,0,Twitter Web App,,2015-01-28 12:10:27,False,19948,1262
970705,tremendous flexibility of synthetic biology mo...,2021-02-25 01:48:52,1364754203663564800,282076470,AgBioWorld,"Professor, biotech guru. I am curious about sc...",False,"Alabama, USA",rt tremendous flexibility of synthetic biology...,"Cold Spring Harbor, NY",...,1364847445717504001,1692248610,JasonWilliamsNY,0,Twitter for iPhone,,2013-08-22 22:53:04,False,75992,4314
970706,the will receive a batch of 600 000 doses of d...,2021-02-25 06:02:00,1364817905364340739,1115874631,CGTNOfficial,CGTN is an international media organization. I...,True,"Beijing, China",rt the will receive a batch of 600 000 doses o...,Republic of Korea,...,1364847445738356738,890916095963430912,Totoyami1,0,Twitter for Android,Philippines,2017-07-28 12:45:24,False,5696,8


In [44]:
clean_df.to_pickle("./cleaned_strip@.pkl")

In [37]:
all_df.shape

(970708, 23)